In [5]:
import numpy as np
import matplotlib.pyplot as plt


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


def grad_desc(func, lr, x, eps):
    points = [x]

    while True:
        prev_x = x
        x = x - lr(Function(lambda a: func.value(x - a * func.grad(x)))) * func.grad(x)
        points.append(x)
        if abs(func.value(x) - func.value(prev_x)) < eps:
            break

    return np.array(points)


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt):
    while abs(a_1 - a_2) >= eps:
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


In [6]:

def func(x):
    return (x[0] - 4) ** 2 / 10 + (5 - x[1]) ** 2


def lr_wrapper(eps, delt):
    return lambda func: dichotomy(func, 0, right_border_calc(func), eps, delt)


print(grad_desc(Function(func), lr_wrapper(0.01, 0.0002), np.array((20., 20.)), 0.001))

[[  20.           20.        ]
 [1000.78030584    9.99203769]
 [ 999.63971547    5.12738946]
 [1000.00647911    5.0409359 ]
 [ 999.99700845    5.00104462]]
